In [1]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier

In [2]:
data = spark.read.csv("/FileStore/tables/dog_food.csv", inferSchema=True, header=True)

In [3]:
data.show()

+---+---+----+---+-------+
 A| B| C| D|Spoiled|
+---+---+----+---+-------+
 4| 2|12.0| 3| 1.0|
 5| 6|12.0| 7| 1.0|
 6| 2|13.0| 6| 1.0|
 4| 2|12.0| 1| 1.0|
 4| 2|12.0| 3| 1.0|
 10| 3|13.0| 9| 1.0|
 8| 5|14.0| 5| 1.0|
 5| 8|12.0| 8| 1.0|
 6| 5|12.0| 9| 1.0|
 3| 3|12.0| 1| 1.0|
 9| 8|11.0| 3| 1.0|
 1| 10|12.0| 3| 1.0|
 1| 5|13.0| 10| 1.0|
 2| 10|12.0| 6| 1.0|
 1| 10|11.0| 4| 1.0|
 5| 3|12.0| 2| 1.0|
 4| 9|11.0| 8| 1.0|
 5| 1|11.0| 1| 1.0|
 4| 9|12.0| 10| 1.0|
 5| 8|10.0| 9| 1.0|
+---+---+----+---+-------+
only showing top 20 rows

In [4]:
assembler = VectorAssembler(inputCols=['A', 'B', 'C', 'D'], outputCol='features')
assembled_data = assembler.transform(data)

In [5]:
assembled_data.show()

+---+---+----+---+-------+-------------------+
 A| B| C| D|Spoiled| features|
+---+---+----+---+-------+-------------------+
 4| 2|12.0| 3| 1.0| [4.0,2.0,12.0,3.0]|
 5| 6|12.0| 7| 1.0| [5.0,6.0,12.0,7.0]|
 6| 2|13.0| 6| 1.0| [6.0,2.0,13.0,6.0]|
 4| 2|12.0| 1| 1.0| [4.0,2.0,12.0,1.0]|
 4| 2|12.0| 3| 1.0| [4.0,2.0,12.0,3.0]|
 10| 3|13.0| 9| 1.0|[10.0,3.0,13.0,9.0]|
 8| 5|14.0| 5| 1.0| [8.0,5.0,14.0,5.0]|
 5| 8|12.0| 8| 1.0| [5.0,8.0,12.0,8.0]|
 6| 5|12.0| 9| 1.0| [6.0,5.0,12.0,9.0]|
 3| 3|12.0| 1| 1.0| [3.0,3.0,12.0,1.0]|
 9| 8|11.0| 3| 1.0| [9.0,8.0,11.0,3.0]|
 1| 10|12.0| 3| 1.0|[1.0,10.0,12.0,3.0]|
 1| 5|13.0| 10| 1.0|[1.0,5.0,13.0,10.0]|
 2| 10|12.0| 6| 1.0|[2.0,10.0,12.0,6.0]|
 1| 10|11.0| 4| 1.0|[1.0,10.0,11.0,4.0]|
 5| 3|12.0| 2| 1.0| [5.0,3.0,12.0,2.0]|
 4| 9|11.0| 8| 1.0| [4.0,9.0,11.0,8.0]|
 5| 1|11.0| 1| 1.0| [5.0,1.0,11.0,1.0]|
 4| 9|12.0| 10| 1.0|[4.0,9.0,12.0,10.0]|
 5| 8|10.0| 9| 1.0| [5.0,8.0,10.0,9.0]|
+---+---+----+---+-------+-------------------+
only showing top 20 rows

In [6]:
final_data = assembled_data.select('features', 'Spoiled')

In [7]:
rdf_class = RandomForestClassifier(featuresCol='features', labelCol='Spoiled')

In [8]:
#no need to divide data into train and test as we only need to understand the variable among A, B, C and D is causing early spoil of the dog food
model_fit = rdf_class.fit(final_data)

In [9]:
#The feature Importances shows that C (2: 0.9413) is highly impacting the output variable Spoiled
model_fit.featureImportances

Out[11]: SparseVector(4, {0: 0.0224, 1: 0.018, 2: 0.9413, 3: 0.0183})